# Load mCLIP from FLAME

## Issues
* !pip install rich
* https://github.com/Lightning-Universe/lightning-bolts/issues/962#issuecomment-1398247929
* !pip install transformers

In [5]:
from src.utils.eval_util import load_mclip_model


mclip_ckpt = '/home/ctq566/motion-diffusion-model/flame/flame_mclip_hml3d_bc.ckpt'
mclip = load_mclip_model(mclip_ckpt)
mclip.to('cuda:0')


/home/ctq566/miniconda3/envs/flame/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/ctq566/miniconda3/envs/flame/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")
/home/ctq566/miniconda3/envs/flame/lib/python3.8/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
  rank_zero_warn(


MCLIP(
  (text_model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm(

In [7]:
import sys
sys.path.append('../HumanML3D/')

from os.path import join as pjoin
from common.skeleton import Skeleton
from common.quaternion import *
from paramUtil import *
import scipy.ndimage.filters as filters


n_raw_offsets = torch.from_numpy(t2m_raw_offsets)
kinematic_chain = t2m_kinematic_chain
# Face direction, r_hip, l_hip, sdr_r, sdr_l
face_joint_indx = [2, 1, 17, 16]


def get_cont6d_params(positions):
    skel = Skeleton(n_raw_offsets, kinematic_chain, "cpu")
    # (seq_len, joints_num, 4)
    quat_params = skel.inverse_kinematics_np(positions, face_joint_indx, smooth_forward=True)

    '''Quaternion to continuous 6D'''
    cont_6d_params = quaternion_to_cont6d_np(quat_params)
    # (seq_len, 4)
    r_rot = quat_params[:, 0].copy()
    #     print(r_rot[0])
    '''Root Linear Velocity'''
    # (seq_len - 1, 3)
    velocity = (positions[1:, 0] - positions[:-1, 0]).copy()
    #     print(r_rot.shape, velocity.shape)
    velocity = qrot_np(r_rot[1:], velocity)
    '''Root Angular Velocity'''
    # (seq_len - 1, 4)
    r_velocity = qmul_np(r_rot[1:], qinv_np(r_rot[:-1]))
    # (seq_len, joints_num, 4)
    return cont_6d_params, r_velocity, velocity, r_rot


In [8]:
from pytorch3d.transforms import axis_angle_to_matrix, matrix_to_rotation_6d
import torch
import torch.nn.functional as F
import numpy as np

def xyz_to_rot6d(xyz):
    assert isinstance(xyz, np.ndarray), "The input should be numpy array"
    assert len(xyz.shape) == 3, "The shape of xyz should be (seq_len, 22, 3)"
    # Motion length, number of joints, feature dimension
    L, J, D = xyz.shape

    assert J == 22 and D == 3, "The shape of xyz should be (seq_len, 22, 3)"
    # Pad to add the missing hand joints and make it 24 joints
    # axis_angles = np.pad(xyz, (0, 0, 0, 2, 0, 0), mode='constant', value=0)

    # Pad to add the missing hand joints and make it 24 joints using numpy
    xyz = np.pad(xyz, ((0, 0), (0, 2), (0, 0)), mode='constant', constant_values=0)
    cont_6d_params, r_velocity, velocity, r_rot = get_cont6d_params(xyz)
    return cont_6d_params


def rot6d_to_flame(rot6d):
    assert isinstance(rot6d, np.ndarray), "The input should be numpy array"
    assert len(rot6d.shape) == 3 and rot6d.shape[1] == 24 and rot6d.shape[2] == 6, "The shape of rot6d should be (seq_len, 24, 6)"
    # The input is (120, 24, 6), make it (120, 147, 1)
    flame = rot6d.reshape(rot6d.shape[0], -1)
    flame = np.lib.pad(flame, ((0, 0), (0, 3)), 'constant', constant_values=0)
    return flame


In [9]:
xyz = np.load('/home/ctq566/motion-diffusion-model/save/p2p_quaternion_2_random/001/sample00_rep00.npy')
rot6d = xyz_to_rot6d(xyz)
flame = rot6d_to_flame(rot6d)

motion = torch.from_numpy(flame).float().to('cuda:0')
motion.unsqueeze(0).permute(0, 2, 1).shape


torch.Size([1, 147, 120])

In [10]:
mclip.motion_encoder.motion_encoder.layers[0].activation_relu_or_gelu

AttributeError: 'TransformerEncoderLayer' object has no attribute 'activation_relu_or_gelu'

In [12]:
text = ['A person is walking in the street.']
mclip.get_features(motion=motion.unsqueeze(0).permute(0, 2, 1), texts=list(text) * 10, motion_length=torch.tensor([motion.shape[0]], device='cuda:0'))

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/ctq566/miniconda3/envs/flame/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_3644672/375418930.py", line 2, in <module>
    mclip.get_features(motion=motion.unsqueeze(0).permute(0, 2, 1), texts=list(text) * 10, motion_length=torch.tensor([motion.shape[0]], device='cuda:0'))
  File "/home/ctq566/flame/src/models/components/mclip.py", line 176, in get_features
    motion_out = self.motion_encoder(motion, motion_length)
  File "/home/ctq566/miniconda3/envs/flame/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    error_msgs.append('size mismatch for {}: copying a param with shape {} from checkpoint, '
  File "/home/ctq566/flame/src/models/components/mclip.py", line 87, in forward
    encoder_out = self.motion_encoder(
  File "/home/ctq566/miniconda3/envs/flame/lib/python3.8/site-packages/torch/

In [1]:
from src.models.components.my_transformer import TransformerEncoder, TransformerEncoderLayer


In [1]:
import torch
from src.models.components.my_transformer import TransformerEncoder, TransformerEncoderLayer

embed_dim = 512
n_heads = 8
dim_feedforward = 2048
dropout = 0.1

encoder_layers = TransformerEncoderLayer(
    d_model=embed_dim,
    nhead=n_heads,
    dim_feedforward=dim_feedforward,
    dropout=dropout,
    batch_first=True,
)

In [2]:
encoder_layers.activation_relu_or_gelu

1